In [ ]:
!git clone https://github.com/facebookresearch/faiss.git

In [ ]:
!pip install faiss-gpu

In [ ]:
import numpy as np
import time
import h5py
import os
import requests
import tempfile
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
### Download Dataset ###
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/nytimes-256-angular.hdf5")
    loc = os.path.join(tmp, "data.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    data_h5py = h5py.File(loc, "r")


In [ ]:
from sklearn.impute import SimpleImputer


xb = np.array(data_h5py['train'], dtype='float32') # database
xq = np.array(data_h5py['test'], dtype='float32') # queries

my_imputer = SimpleImputer(missing_values = 0, strategy = 'most_frequent') ### Fill the missing values to avoid the NaN during normalization
xb = my_imputer.fit_transform(xb)
xq = my_imputer.fit_transform(xq)

xb = xb / np.linalg.norm(xb, axis=1)[:, np.newaxis]
xq = xq / np.linalg.norm(xq, axis=1)[:, np.newaxis]

nq, d = xq.shape

print(xb.shape)
print(xq.shape)
print(nq, d)

In [ ]:
np.save('catalog.npy', xb)
np.save('queries.npy', xq)

#Set the target reduced dimension for DR approach

In [ ]:
target_dim = 32

#PCA

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(xb)

pca =  PCA(n_components = target_dim)
tr = pca.fit_transform(X_train)

xb = pca.transform(X_train)
xq = pca.transform(xq)


xb = xb / np.linalg.norm(xb, axis=1)[:, np.newaxis]
xq = xq / np.linalg.norm(xq, axis=1)[:, np.newaxis]

nq, d = xq.shape

print(xb.shape)
print(xq.shape)
print(nq, d)



#Algo

In [ ]:
train_embed = xb
test_embed_1 = xb
test_embed_2 = xq

# PCA to get Top Components
pca =  PCA(n_components = train_embed.shape[1])
X_train = train_embed - np.mean(train_embed)
X_fit = pca.fit_transform(X_train)
U1 = pca.components_

z = []

# Removing Projections on Top Components
for i, x in enumerate(X_train):
  for u in U1[0:7]:        
          x = x - np.dot(u.transpose(),x) * u 
  z.append(x)

z = np.asarray(z)

# PCA Dim Reduction
pca =  PCA(n_components = target_dim)
X_train = z - np.mean(z)
X_train_new_final = pca.fit_transform(X_train)
X_new_final_1 = pca.transform(test_embed_1)
X_new_final_2 = pca.transform(test_embed_2)


# PCA to do Post-Processing Again
pca =  PCA(n_components = target_dim)
X_new = X_train_new_final - np.mean(X_train_new_final)
X_train_new = pca.fit_transform(X_new)

X_new_1 = pca.transform(X_new_final_1)
X_new_2 = pca.transform(X_new_final_2)

Ufit = pca.components_

X_new_final_1 = X_new_final_1 - np.mean(X_new_final_1)
X_new_final_2 = X_new_final_2 - np.mean(X_new_final_2)

for i in range(len(X_new_final_1)):
  for u in Ufit[0:7]:
    X_new_final_1[i] = X_new_final_1[i] - np.dot(u.transpose(), X_new_final_1[i]) * u 


for i in range(len(X_new_final_2)):
  for u in Ufit[0:7]:
    X_new_final_2[i] = X_new_final_2[i] - np.dot(u.transpose(), X_new_final_2[i]) * u 

xb = X_new_final_1
xq = X_new_final_2

xb = xb / np.linalg.norm(xb, axis=1)[:, np.newaxis]
xq = xq / np.linalg.norm(xq, axis=1)[:, np.newaxis]

nq, d = xq.shape


print(xb.shape)
print(xq.shape)
print(nq, d)

#UMAP

In [ ]:
pip install umap-learn

In [ ]:
import umap

trans = umap.UMAP(n_neighbors=15, n_components=target_dim, metric='euclidean').fit(xb) # euclidean
xb = trans.transform(xb)
xq = trans.transform(xq)

xb = xb / np.linalg.norm(xb, axis=1)[:, np.newaxis]
xq = xq / np.linalg.norm(xq, axis=1)[:, np.newaxis]

nq, d = xq.shape

print(xb.shape)
print(xq.shape)
print(nq, d)

#EmbedTextNet

In [ ]:
# Please check !python EmbedTextNet_models.py -h to get information of the argparse in each py file
# Before running this cell, you should upload EmbedTextNet_models.py in appropriate location
!python EmbedTextNet_models.py --train-embed catalog.npy --test-embed queries.npy --target-dim 64

In [ ]:
xb = np.load('EmbedTextNet_catalog.npy')
xq = np.load('EmbedTextNet_queries.npy')
nq, d = xq.shape

print(xb.shape)
print(xq.shape)
print(nq, d)

#Evaluation

In [ ]:
def evaluate(index, k, xq):

    gt = data_h5py['neighbors'][:, :k]

    t0 = time.time()
    D, I = index.search(xq, k)
    t1 = time.time()

    missing_rate = (I == -1).sum() / float(k * nq)
    recall_at_1 = (I == gt[:, :1]).sum() / float(nq)

    print("%7.3f ms per query, R@1 %.4f, missing rate %.4f" % (
        (t1 - t0) * 1000.0 / nq, recall_at_1, missing_rate))


In [ ]:
# Run if you want to reset the Faiss index
index.reset()

In [ ]:
import faiss

print("Testing HNSW Flat")

index = faiss.IndexHNSWFlat(d, 12)

start_time = time.time()
index.hnsw.efConstruction = 120 

print("Add")
index.verbose = True
index.add(xb)

print('Total size of index:', index.ntotal)
idx_time = time.time() - start_time
print('Indexing time:', idx_time)

print("Search")
for efSearch in [16]:
    print("EFSearch", efSearch, end=' ')
    index.hnsw.efSearch = efSearch
    evaluate(index, 10, xq)